In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [16]:
#CREATE DATAFRAME FROM CSV FILE
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [17]:
#Function to remove spaces from the beginning and end of the string
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df

#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

+------+-----------+-------------------+-------------------+--------+--------------------+-------------+------+------------+----------+----------+--------+----+
|Delito|  SubDelito|              Fecha|               Hora| Victima|          SubVictima|         Edad|Genero|Nacionalidad| Provincia|    Canton|Distrito|_c12|
+------+-----------+-------------------+-------------------+--------+--------------------+-------------+------+------------+----------+----------+--------+----+
|ASALTO|ARMA BLANCA|2021-06-03 00:00:00|00:00:00 - 02:59:59|VEHICULO|SERVICIO PUBLICO/...|Mayor de edad|HOMBRE|  COSTA RICA|  san jose|alajuelita|    null|null|
|ASALTO|ARMA BLANCA|2021-06-10 00:00:00|15:00:00 - 17:59:59| PERSONA|    PEATON [PERSONA]|Mayor de edad| MUJER|  COSTA RICA|  san jose|    escazu|    null|null|
|ASALTO|ARMA BLANCA|2021-06-14 00:00:00|09:00:00 - 11:59:59| PERSONA|    PEATON [PERSONA]|Mayor de edad|HOMBRE|  COSTA RICA|guanacaste|    nicoya|    null|null|
|ASALTO|ARMA BLANCA|2021-06-14 00:

In [18]:
#Function find list the not match values in the dataframes
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            for row in df1.select(col).distinct().collect():
                if not df2.filter(df2['Provincia, cantón y distrito'] == row[col]).collect():
                    non_matches.append(row[col])
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)


print(non_matches_oij)


['san jose', 'limon', 'islas', 'desconocido', 'pococi', 'rio cuarto', 'guacimo', 'belen', 'la union', 'desconocido', 'puerto jim&#201;nez', 'leon cortes', 'poas', 'tarrazu', 'san jose', 'san ramon', 'aserri', 'vasquez de coronado', 'sarch&#205;', 'paraiso', 'tilaran', 'canas', 'limon', 'perez zeledon', 'jimenez', 'tibas', 'sarapiqui', 'escazu', 'santa barbara', 'monteverde', None]


In [19]:
#Function to replace the accents in column Provincia, cantón y distrito in inec_df
def replace_accents(df):
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'á', 'a'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'é', 'e'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'í', 'i'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ó', 'o'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ú', 'u'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ñ', 'n'))
    return df

inec_df = replace_accents(inec_df)
nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)


['islas', 'desconocido', 'desconocido', 'puerto jim&#201;nez', 'leon cortes', 'vasquez de coronado', 'sarch&#205;', 'monteverde', None]
